In [2]:
from dotenv import dotenv_values

In [4]:
from langchain.llms import OpenAI, HuggingFaceHub, HuggingFaceTextGenInference

In [6]:
llm = OpenAI(api_key = API_KEY, base_url = BASE_URL, model_name = "gpt-3.5-turbo")

NameError: name 'API_KEY' is not defined

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings

In [ ]:
chat = ChatOpenAI(
    api_key = API_KEY,
    base_url = BASE_URL,
    model_name = "gpt-3.5-turbo"
)

ans = chat.invoke("帮我写一个关于计算机的笑话，不要超过100个字")
print(ans.content)

In [ ]:
chat.batch(["帮我写一个关于猫的笑话，不要超过100个字"])

In [10]:
from langchain.prompts import PromptTemplate

In [12]:
template = PromptTempalte.from_template(
    "请帮我写一个关于{topic}的笑话。输出遵循下面这个格式：{format}",
    partial_variables = {"format": "仅仅输出一个笑话，不要输出其他任何内容"}
)

pp = prompt.format(topic = "猫")
print(pp)

NameError: name 'PromptTempalte' is not defined

In [ ]:
prompt_text = PromptTemplate.from_template("写一个关于{topic}的笑话")
prompt_text.format(topic = "dog")


In [ ]:
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
examples =[
    {"word":"高", "antonym":"矮"},
    {"word":"胖","antonym":"瘦"},
    ]
example_template ="""
词语: {word}
反义词: {antonym}\n
"""
example_prompt = PromptTempate(
    input_variables = ["word", "antonym"],
    template = example_template,
    few_shot_prompt = FewShotPromptTemplate(
        examples=examples[:1],
    example_prompt=example_prompt,
    prefix="给出输入词语的反义词",
    suffix="词语: {input}\n反义词:",
    input_variables=["input"],
    example_separator="\n",    
    )
)

In [14]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

In [ ]:
tools = load_tools(["wikipedia", "kkn-math"], llm = llm)
agent = initialize_agent(
    tools,
    llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True
)

agent.run("马斯克的生日是哪天")

In [16]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.agents import AgentExecutor

from langchain.agents import tool

In [ ]:
@tool
def get_word_length(word:str) -> int:
    return len(word)

tools = [get_word_length]

prompt = ChatPromptTemplate.form_messages(
    [
        (
            "system",
            "you are very powerful assistant, but do not know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name = "agent_scratchpad"),
    ]
)

llm_with_tools = chat.bind(function = [format_tool_to_openai_function(t) for t in tools])

agent = (
    {
        "input": lambda x:x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        )
    }
    | prompt
    |llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

agent_executor = AgentExecutor(agent = agent, tools = tools, verbose = True)
agent_executor.invoke({"input": "how many letters inthe word educa" })

In [ ]:
# 推荐阅读 openai 怎么调用 function
import openai
import json
from enum import Enum

class BaseTool(Enum):
    Bookkeeping = "record_price"
    RecordingTask = "record_task"

# record_price是用来给Function Calling调用的函数，
# 这个函数接收两个必填的参数，category类目（string类型），count 数量（int类型）
def record_price(category, count):
    print(category, count)
    print("调用获取实时物价的 API")
    return count*12

def funtion_call_conversation(message):
    messages = [
      {"role": "user", "content": "今天买了一斤肉，花了多少钱"}
    ]
    response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages = message,
    temperature=0,
    functions=[
      {
        "name": BaseTool.Bookkeeping.value,
        "description": "返回物品的数量",
        "parameters": {
          "type": "object",
          "properties": {
            "category": {"type": "string","description": "类目",},
            "count": {"type": "integer", "description": "数量"},
          },
          "required": ["category","count"],
        },
      }
    ],
    function_call="auto",
    )
    message = response["choices"][0]["message"]
    print(message)
    if(message.get("function_call")):
        function_name = message["function_call"]["name"]
        if function_name == BaseTool.Bookkeeping.value:
            arguments = json.loads(message["function_call"]["arguments"])
            price = record_price(arguments.get('category'), arguments.get('count'))
            messages.append(message)
            messages.append({"role": "function", "name": BaseTool.Bookkeeping.value, "content": price})
            print(messages)
            role_function_conversation(messages)

def role_function_conversation(message):
    response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages = message,
    temperature=0,
    # name字段表示要调用的函数名，description表示函数描述（让 LLM 读得懂的函数说明）
    # paramters是一个符合JSON Schema格式的对象，用来描述这个函数的入参信息
    functions=[
      {
        "name": BaseTool.Bookkeeping.value,
        "description": "返回物品的数量",
        "parameters": {
          "type": "object",
          "properties": {
            "category": {"type": "string","description": "类目",},
            "count": {"type": "integer", "description": "数量"},
          },
          "required": ["category","count"],
        },
      }
    ],
    function_call="auto",
    )
    message = response["choices"][0]["message"].get("content")
    print(message)
    

if __name__ == "__main__":
    funtion_call_conversation()